In [1]:
import logging
import sys

fx_spot_tickers = [
    "EURUSD=X",
    "GBPUSD=X",
    "JPYUSD=X",
    "AUDUSD=X",
    "NZDUSD=X",
    "USDCHF=X",
    "USDCAD=X",
    "USDSEK=X",
    "USDNOK=X",
]
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

In [ ]:
import yfinance as yf
from importlib.resources import files
import backtest_lib.examples
import pickle as pkl

data = yf.download(
    tickers=fx_spot_tickers,
    start="2015-01-01",
    end="2025-01-01",
    interval="1d",
    auto_adjust=True,
)

pkl.dump(data, files(backtest_lib.examples).joinpath("fx_spot.pkl").open("wb"))

In [3]:
import backtest_lib as btl

px_flat = data["Close"].rename_axis(index="date", columns=None).ffill().reset_index()

market_view = btl.MarketView(prices=px_flat)

In [4]:
universe = tuple(fx_spot_tickers)


def index_strategy(universe, current_portfolio, market, ctx):
    return btl.Decision(current_portfolio)

In [5]:
from backtest_lib.portfolio import uniform_portfolio
from backtest_lib.backtest import Backtest

backtest = Backtest(
    market_view=market_view,
    universe=universe,
    strategy=index_strategy,
    initial_portfolio=uniform_portfolio(universe),
)

In [6]:
results = backtest.run()

In [ ]:
results.total_growth